In [1]:
# importing all the required modules

from importlib import reload
import nltk
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import os
import errno
import string
import collections
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from prettytable import PrettyTable
import time
from datetime import timedelta
from sklearn.metrics import recall_score,precision_score,average_precision_score,f1_score
from sklearn.metrics import accuracy_score,roc_auc_score,mean_absolute_error,cohen_kappa_score
from sklearn.preprocessing import label_binarize,LabelEncoder
from imblearn.metrics import geometric_mean_score
from sklearn import metrics
from sklearn.metrics.cluster import homogeneity_score,completeness_score
import statistics
import math
import sklearn.metrics 
from sklearn.model_selection import KFold,StratifiedKFold
import csv
from sklearn.model_selection import train_test_split
from collections import Counter
import math
from scipy.spatial import distance
import statistics
from scipy.stats import pearsonr,entropy,mode
from scipy.io import arff
from sklearn.metrics import pairwise_distances
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestNeighbors
from sklearn.datasets import load_iris
import warnings
import random
from statistics import mode
random.seed(0)
warnings.filterwarnings('ignore')

def loadNonIRDatasets(dataset):
    documents,labels=[],[]
    fname=dataset+'.csv'
    with open(fname, 'r') as file:
        reader = csv.reader(file, delimiter = ',')
        for row in reader:
            if len(row)>0:
                if '?' in row:
                    continue
                doc=row[0:len(row)-2]
                try:
                    doc=[abs(float(d)) for d in doc]
                except:
                    continue
                documents.append(doc)
                #print(row)
                labels.append(row[len(row)-1])

    return documents,labels

def Cosine(a, b):#distance
    return distance.cosine(a,b)

def Euclidean(a, b):#distance
    return distance.euclidean(a,b)

def PrintDetails(metric="smtp",measure= "Accuracy",Arr=None,kList=[5,15,30,45],train_time=0,test_time=0,std=[]):
    x = PrettyTable()
    #kList=[1,3,5,9,15,30,45]#,70,90,120]
    if std==[]:
        x.field_names = ["k",measure,"Train Time","Test time","Total Time"]
    else:
        x.field_names = ["Split",measure,"std","Train Time","Test time","Total Time"]
        
    print("metric",metric)
    print("measure",measure)
    tables.write(("metric\t"+str(metric)+"\n").encode())
    tables.write(("measure\t"+str(measure)+"\n").encode())
    average=0
    total_time=np.add(train_time,test_time)
    for i in range(0,len(Arr)):
        print(train_time[i])
        tr_time=str(timedelta(seconds=(train_time[i])))
        ts_time=str(timedelta(seconds=(test_time[i])))
        
        tot_time=str(timedelta(seconds=(total_time[i])))
        if std==[]:
            x.add_row([str(kList[i]),round(Arr[i],4),tr_time,ts_time,tot_time]) 
        else:
            x.add_row([str(kList[i]),round(Arr[i],4),round(std[i],4),tr_time,ts_time,tot_time]) 
            
        average+=Arr[i]
    if std==[]:
        x.add_row(["Average",round((average/len(Arr)),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),np.std(train_time),
                   np.std(test_time),np.std(total_time)])   
    else:
        x.add_row(["Average",round((average/len(Arr)),4),round(np.mean(std),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),round(np.std(std),4),
                   round(np.std(train_time),10),
                   round(np.std(test_time),10),
                   round(np.std(total_time),10)])
        
    x.padding_width =1
    tables.write(str(x).encode())
    print(x)

def findDI(train_data,train_labels,metric,k):
    clss=set(train_labels)
    func=globals()[metric]
    radius={}
    th=0.5
    for cl in clss:
        indices=[i for i in range(0,len(train_labels)) if train_labels[i]==cl]
        elements=train_data[indices,:]
        distMat=pairwise_distances(elements,elements,metric=func)
        radius[cl]=np.mean(distMat)
    weights=[]
   
    distances=pairwise_distances(train_data,train_data,metric=func)
    for i in range(0,train_data.shape[0]):
        r=radius[train_labels[i]]
        ID=0
        val=0
        c=1
        for nn,neigh in enumerate(train_labels):
            if distances[i][nn]<r and i!=nn:
                c=c+1
                if neigh==train_labels[i]:
                    val=val+1
        ID=val/c
        ##############find weights using validity ###############
        #w=np.sum(val*(1/(distances[i]+alpha)))
        weights.append(ID)
    return weights
    

        
def classify(k=3,metric="euclidean",termOccurance=None, docOccurance=None):
    accuracy=[]
    precision=[]
    Recall=[]
    fMeasure=[]
    mprecision=[]
    mRecall=[]
    mfMeasure=[]
    roc=[]
    y_pred=[]
    mae=[]
    kappa=[]
    averageMeanPrecison=[]
    func=globals()[metric]
    kList=[5,15,30,45]
    var=np.zeros(train_data.shape[1])
    train_time=[]
    test_time=[]
    for k in kList:
        time1=time.time()
        print("k",k)
        weights=findDI(train_data,train_labels,metric,k)
        classifier = NearestNeighbors(n_neighbors=k,metric=func)
        if k>len(train_labels):
             classifier = NearestNeighbors(n_neighbors=len(train_labels),metric=func)
        classifier.fit(train_data,train_labels)
        distances,neighVals=classifier.kneighbors(test_data,return_distance=True)
        pred=[]
        time2=time.time()
        for i in range(0,test_data.shape[0]):
            neighs=np.array(neighVals[i,:],dtype=int)
            #print(neighs)
            neighLabels = [train_labels[i] for i in neighs]
            dLabels=set(neighLabels)
            weightSum=0
            maxweight=0
            predLabel=''
            for lbl in dLabels:
                weightSum=0
                #print(neighs)
                weightSum=np.mean([weights[ii]/distances[i][j] for j,ii in enumerate(neighs) if train_labels[ii]==lbl])
                #print(weightSum)
                if weightSum>=maxweight:
                    maxweight=weightSum
                    predLabel=lbl
            pred.append(predLabel)  
        y_pred.append(pred)
        time3=time.time()
        train_time.append(time2-time1)
        test_time.append(time3-time2)
 
   
    class_dict={}
    classes=list(set(labels))
    for i in range(0,len(classes)):
        class_dict[classes[i]]=i
    print(class_dict)
    num_testLabels=[]
    for i in range(0,len(test_labels)):
        num_testLabels.append(class_dict[test_labels[i]])
       
    for i in range(0,len(y_pred)):
        test_labels1=list(test_labels)
        #Accuracy
       
        accuracy.append(accuracy_score(test_labels, y_pred[i]))
        
        #Precison
        precision.append(precision_score(test_labels, y_pred[i],average='macro'))
         
        #Recall
        Recall.append(recall_score(test_labels, y_pred[i],average='macro'))
           
        #f measure
        fMeasure.append(f1_score(test_labels, y_pred[i],average='macro'))
        
        #Precison
        mprecision.append(precision_score(test_labels, y_pred[i],average='micro'))
         
        #Recall
        mRecall.append(recall_score(test_labels, y_pred[i],average='micro'))
           
        #f measure
        mfMeasure.append(f1_score(test_labels, y_pred[i],average='micro'))
        
        kappa.append(cohen_kappa_score(test_labels, y_pred[i]))
      
        test = label_binarize(test_labels1, classes=categories).reshape((-1))
        pred = label_binarize( y_pred[i], classes=categories).reshape((-1))
        
        #roc
        try:
            roc.append(roc_auc_score(test,pred))
        except:
            roc.append(-1)
            
        #Average Mean Precision
        #y_val_true, val_pred = y_val_true.reshape((-1)), val_pred.reshape((-1))
        averageMeanPrecison.append(average_precision_score(test, pred))
        #MAE
        num_pred=[]
        for j in range(0,len(y_pred[i])):
            num_pred.append(class_dict[y_pred[i][j]])
        mae.append(mean_absolute_error(num_testLabels, num_pred))
        
        #PrintDetails(metric=metric,time=str(timedelta(seconds=(time2-time1))),measure="Average Mean Precison",Arr=averageMeanPrecison)


    
   #Accuracy
    PrintDetails(metric=metric,measure="Accuracy",Arr=accuracy,train_time=train_time,test_time=test_time)
    
    #Precison
    PrintDetails(metric=metric,measure="Precision",Arr=precision,train_time=train_time,test_time=test_time)
    #Precison
    PrintDetails(metric=metric,measure="micro Precision",Arr=mprecision,train_time=train_time,test_time=test_time)
    #Recall
    PrintDetails(metric=metric,measure="Recall",Arr=Recall,train_time=train_time,test_time=test_time)
     #Recall
    PrintDetails(metric=metric,measure="micro Recall",Arr=mRecall,train_time=train_time,test_time=test_time)
    #f measure
    PrintDetails(metric=metric,measure="F Measure",Arr=fMeasure,train_time=train_time,test_time=test_time)
    
    #f measure
    PrintDetails(metric=metric,measure="micro F Measure",Arr=mfMeasure,train_time=train_time,test_time=test_time)
    
    #roc
    PrintDetails(metric=metric,measure="ROC",Arr=roc,train_time=train_time,test_time=test_time)
    #Average Mean Precision
    PrintDetails(metric=metric,measure="Average Mean Precison",Arr=averageMeanPrecison,train_time=train_time,test_time=test_time)
    #MAE
    PrintDetails(metric=metric,measure="MAE",Arr=mae,train_time=train_time,test_time=test_time)
  
    accuracy_avg.append(accuracy)
    precision_avg.append(precision)
    recall_avg.append(Recall)
    macroFMeasure_avg.append(fMeasure)
    mprecision_avg.append(mprecision)
    mrecall_avg.append(mRecall)
    mFMeasure_avg.append(mfMeasure)
    roc_avg.append(roc)
    AMP_avg.append(averageMeanPrecison)
    mae_avg.append(mae)
    avg_test_time.append(test_time)
    avg_train_time.append(train_time)
    
import glob
datasets=[]
for file_name in glob.glob("Datasets April\\"+'*.csv'):
    file=os.path.split(file_name)[1]
    datasets.append(os.path.splitext(file)[0])

    
for dataset in datasets:
    fname="Datasets April\\"+dataset
    print(fname.encode())
    arr,labels=loadNonIRDatasets(fname)
    arr=np.array(arr)
    labels=np.array(labels)
    labels=list(labels)
    le = LabelEncoder()
    labels=le.fit_transform(labels)
    categories=list(set(labels))
    measures=["Euclidean"]
    global w
    w=0
    global Weights
    for met in measures:
        n_split=1
        labels=np.array(labels)
        path = "DKNN Original Corrected\\April Task1\\Cancelled Datasets"
        if not os.path.exists(path):
            os.makedirs(path)
        fname=path+'\\table_'+dataset+'_'+met+'_DKNN_original_corrected.txt'
        tables = open(fname, 'wb')
        k_splits=10
        accuracy_avg=[]
        precision_avg=[]
        recall_avg=[]
        macroFMeasure_avg=[]
        mprecision_avg=[]
        mrecall_avg=[]
        mFMeasure_avg=[]
        roc_avg=[]
        AMP_avg=[]
        mae_avg=[]
        avg_test_time=[]
        avg_train_time=[]
        kf = StratifiedKFold(n_splits=k_splits)
        for trn_ind, tst_ind in kf.split(arr,labels):
            xTrain, xTest = arr[trn_ind], arr[tst_ind]
            yTrain, yTest = labels[trn_ind], labels[tst_ind]
            train_data=np.array(xTrain)
            test_data=np.array(xTest)
            train_labels=np.array(yTrain)
            test_labels=np.array(yTest)
            allData=arr
            termOcc=[]
            docOcc=[]
            tables.write("\n*************".encode())
            tables.write((" Split\t"+str(n_split)).encode())
            tables.write(" ***********\n".encode())
            n_split=n_split+1
            k=1
            print("###############")
            Weights=np.ones(train_data.shape[0])
            classify(k=k,metric=met,termOccurance=termOcc,docOccurance=docOcc)
        list1=[5,15,30,45]
        tables.write("\n************* Average Results ***********\n".encode())
        acc_std=np.std(np.array(accuracy_avg),axis=0)
        accuracy_avg=np.mean(np.array(accuracy_avg),axis=0)
        
        pr_std=np.std(np.array(precision_avg),axis=0)
        precision_avg=np.mean(np.array(precision_avg),axis=0)
        
        mpr_std=np.std(np.array(mprecision_avg),axis=0)
        mprecision_avg=np.mean(np.array(mprecision_avg),axis=0)
        
        rcl_std=np.std(np.array(recall_avg),axis=0)
        recall_avg=np.mean(np.array(recall_avg),axis=0)
        
        mrcl_std=np.std(np.array(mrecall_avg),axis=0)
        mrecall_avg=np.mean(np.array(mrecall_avg),axis=0)
        
        F_std=np.std(np.array(macroFMeasure_avg),axis=0)
        macroFMeasure_avg=np.mean(np.array(macroFMeasure_avg),axis=0)
        
        mF_std=np.std(np.array(mFMeasure_avg),axis=0)
        mFMeasure_avg=np.mean(np.array(mFMeasure_avg),axis=0)
        
        roc_std=np.std(np.array(roc_avg),axis=0)
        roc_avg=np.mean(np.array(roc_avg),axis=0)
        
        amp_std=np.std(np.array(AMP_avg),axis=0)
        AMP_avg=np.mean(np.array(AMP_avg),axis=0)
        
        mae_std=np.std(np.array(mae_avg),axis=0)
        mae_avg=np.mean(np.array(mae_avg),axis=0)
        
        avg_train_time=np.mean(np.array(avg_train_time),axis=0)
        avg_test_time=np.mean(np.array(avg_test_time),axis=0)
        #Accuracy
        PrintDetails(metric=met,measure="Accuracy",Arr=accuracy_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=acc_std)
        
        #Precision
        PrintDetails(metric=met,measure="Precision",Arr=precision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=pr_std)
        
        #Precision
        PrintDetails(metric=met,measure="Micro Precision",Arr=mprecision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mpr_std)


        #Recall
        PrintDetails(metric=met,measure="Recall",Arr=recall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=rcl_std)
        
         #Recall
        PrintDetails(metric=met,measure="Micro Recall",Arr=mrecall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mrcl_std)


        #f measure
        PrintDetails(metric=met,measure="F Measure",Arr=macroFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=F_std)
        
       
       
        #f measure
        PrintDetails(metric=met,measure="Micro F Measure",Arr=mFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mF_std)


        #roc
        PrintDetails(metric=met,measure="ROC",Arr=roc_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=roc_std)

        #average Mean precision
        PrintDetails(metric=met,measure="AMP",Arr=AMP_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=amp_std)
         
        #MAE
        PrintDetails(metric=met,measure="MAE",Arr=mae_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mae_std)
        
        tables.close()
        print("###############")





b'Datasets April\\1_lung-cancer'
###############
k 5
k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.00495457649230957
0.003989219665527344
0.003988981246948242
0.004986763000488281
+----------+----------+-----------------------+-----------------------+-----------------------+
|    k     | Accuracy |       Train Time      |       Test time       |       Total Time      |
+----------+----------+-----------------------+-----------------------+-----------------------+
|    5     |   0.75   |     0:00:00.004955    |        0:00:00        |     0:00:00.004955    |
|    15    |   0.75   |     0:00:00.003989    |     0:00:00.000998    |     0:00:00.004987    |
|    30    |   0.75   |     0:00:00.003989    |        0:00:00        |     0:00:00.003989    |
|    45    |   0.75   |     0:00:00.004987    |        0:00:00        |     0:00:00.004987    |
| Average  |   0.75   |     0:00:00.004480    |     0:00:00.000249    |     0:00:00.004729    |
| Std Dev. |   0.0    | 0.00049091

metric Euclidean
measure Accuracy
0.00498652458190918
0.004986763000488281
0.003989219665527344
0.003988981246948242
+----------+----------+-----------------------+-----------------------+----------------------+
|    k     | Accuracy |       Train Time      |       Test time       |      Total Time      |
+----------+----------+-----------------------+-----------------------+----------------------+
|    5     |   1.0    |     0:00:00.004987    |        0:00:00        |    0:00:00.004987    |
|    15    |  0.6667  |     0:00:00.004987    |        0:00:00        |    0:00:00.004987    |
|    30    |  0.6667  |     0:00:00.003989    |     0:00:00.000998    |    0:00:00.004987    |
|    45    |  0.6667  |     0:00:00.003989    |     0:00:00.000997    |    0:00:00.004986    |
| Average  |   0.75   |     0:00:00.004488    |     0:00:00.000499    |    0:00:00.004987    |
| Std Dev. |  0.1443  | 0.0004987716817263205 | 0.0004987120699594706 | 1.97686242482388e-07 |
+----------+----------+-----

k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.003989219665527344
0.004986763000488281
0.00498652458190918
0.004986763000488281
+----------+----------+------------------------+-----------------------+-----------------------+
|    k     | Accuracy |       Train Time       |       Test time       |       Total Time      |
+----------+----------+------------------------+-----------------------+-----------------------+
|    5     |  0.6667  |     0:00:00.003989     |        0:00:00        |     0:00:00.003989    |
|    15    |  0.6667  |     0:00:00.004987     |     0:00:00.000998    |     0:00:00.005984    |
|    30    |  0.6667  |     0:00:00.004987     |        0:00:00        |     0:00:00.004987    |
|    45    |  0.6667  |     0:00:00.004987     |        0:00:00        |     0:00:00.004987    |
| Average  |  0.6667  |     0:00:00.004737     |     0:00:00.000249    |     0:00:00.004987    |
| Std Dev. |   0.0    | 0.00043191453293564026 | 0.0004319489347260107 | 0.00070536966423

k 15
k 30
k 45
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12}
metric Euclidean
measure Accuracy
0.8497264385223389
0.8417408466339111
0.8228034973144531
0.8297834396362305
+----------+----------+---------------------+-----------------------+----------------------+
|    k     | Accuracy |      Train Time     |       Test time       |      Total Time      |
+----------+----------+---------------------+-----------------------+----------------------+
|    5     |  0.587   |    0:00:00.849726   |     0:00:00.001000    |    0:00:00.850726    |
|    15    |  0.5217  |    0:00:00.841741   |     0:00:00.002992    |    0:00:00.844733    |
|    30    |  0.5217  |    0:00:00.822803   |     0:00:00.003983    |    0:00:00.826787    |
|    45    |  0.5217  |    0:00:00.829783   |     0:00:00.004993    |    0:00:00.834776    |
| Average  |  0.538   |    0:00:00.836014   |     0:00:00.003242    |    0:00:00.839256    |
| Std Dev. |  0.0282  | 0.01041831326869246 | 

k 15
k 30
k 45
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12}
metric Euclidean
measure Accuracy
0.8207716941833496
0.8278164863586426
0.827812671661377
0.8257908821105957
+----------+----------+----------------------+-----------------------+----------------------+
|    k     | Accuracy |      Train Time      |       Test time       |      Total Time      |
+----------+----------+----------------------+-----------------------+----------------------+
|    5     |  0.5778  |    0:00:00.820772    |     0:00:00.000998    |    0:00:00.821769    |
|    15    |  0.5333  |    0:00:00.827816    |     0:00:00.002992    |    0:00:00.830808    |
|    30    |  0.5333  |    0:00:00.827813    |     0:00:00.003989    |    0:00:00.831802    |
|    45    |  0.5333  |    0:00:00.825791    |     0:00:00.005984    |    0:00:00.831775    |
| Average  |  0.5444  |    0:00:00.825548    |     0:00:00.003491    |    0:00:00.829039    |
| Std Dev. |  0.0192  | 0.0028786653612

...

k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
1.2556111812591553
1.278613805770874
1.2336654663085938
1.283567190170288
+----------+----------+----------------------+-----------------------+---------------------+
|    k     | Accuracy |      Train Time      |       Test time       |      Total Time     |
+----------+----------+----------------------+-----------------------+---------------------+
|    5     |   1.0    |    0:00:01.255611    |     0:00:00.000997    |    0:00:01.256608   |
|    15    |  0.9636  |    0:00:01.278614    |     0:00:00.001994    |    0:00:01.280608   |
|    30    |  0.8545  |    0:00:01.233665    |     0:00:00.001995    |    0:00:01.235660   |
|    45    |  0.8909  |    0:00:01.283567    |     0:00:00.002993    |    0:00:01.286560   |
| Average  |  0.9273  |    0:00:01.262864    |     0:00:00.001995    |    0:00:01.264859   |
| Std Dev. |  0.0575  | 0.019886152303725223 | 0.0007054540359884858 | 0.02024591859031434 |
+----------+----------+----

k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
1.2688157558441162
1.2895457744598389
1.2536413669586182
1.2895607948303223
+----------+----------+----------------------+---------------------+---------------------+
|    k     | Accuracy |      Train Time      |      Test time      |      Total Time     |
+----------+----------+----------------------+---------------------+---------------------+
|    5     |  0.8909  |    0:00:01.268816    |    0:00:00.000999   |    0:00:01.269815   |
|    15    |  0.7818  |    0:00:01.289546    |    0:00:00.001999   |    0:00:01.291545   |
|    30    |  0.7818  |    0:00:01.253641    |    0:00:00.001993   |    0:00:01.255634   |
|    45    |  0.7818  |    0:00:01.289561    |    0:00:00.003984   |    0:00:01.293545   |
| Average  |  0.8091  |    0:00:01.275391    |    0:00:00.002244   |    0:00:01.277635   |
| Std Dev. |  0.0472  | 0.015144480006448595 | 0.00108415113672797 | 0.01574638457466172 |
+----------+----------+--------------------

k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
1.254643440246582
1.2496881484985352
1.2217295169830322
1.2615995407104492
+----------+----------+----------------------+-----------------------+----------------------+
|    k     | Accuracy |      Train Time      |       Test time       |      Total Time      |
+----------+----------+----------------------+-----------------------+----------------------+
|    5     |  0.7818  |    0:00:01.254643    |     0:00:00.000997    |    0:00:01.255641    |
|    15    |  0.8182  |    0:00:01.249688    |     0:00:00.001997    |    0:00:01.251685    |
|    30    |  0.8545  |    0:00:01.221730    |     0:00:00.002987    |    0:00:01.224716    |
|    45    |  0.9273  |    0:00:01.261600    |     0:00:00.003020    |    0:00:01.264619    |
| Average  |  0.8455  |    0:00:01.246915    |     0:00:00.002250    |    0:00:01.249165    |
| Std Dev. |  0.0538  | 0.015144004026431209 | 0.0008319631840222275 | 0.014873193942967367 |
+----------+-----

k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
1.229717493057251
1.278604507446289
1.2661664485931396
1.297530174255371
+----------+----------+----------------------+-----------------------+----------------------+
|    k     | Accuracy |      Train Time      |       Test time       |      Total Time      |
+----------+----------+----------------------+-----------------------+----------------------+
|    5     |  0.9273  |    0:00:01.229717    |     0:00:00.001995    |    0:00:01.231713    |
|    15    |  0.7818  |    0:00:01.278605    |     0:00:00.001995    |    0:00:01.280600    |
|    30    |  0.7818  |    0:00:01.266166    |     0:00:00.002992    |    0:00:01.269159    |
|    45    |  0.8182  |    0:00:01.297530    |     0:00:00.003989    |    0:00:01.301520    |
| Average  |  0.8273  |    0:00:01.268005    |     0:00:00.002743    |    0:00:01.270748    |
| Std Dev. |  0.0596  | 0.024765898588618274 | 0.0008268137250067925 | 0.025348754828746982 |
+----------+-------

k 15
k 30
k 45
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
1.2287147045135498
1.2736618518829346
1.2508184909820557
1.2865633964538574
+----------+----------+---------------------+-----------------------+----------------------+
|    k     | Accuracy |      Train Time     |       Test time       |      Total Time      |
+----------+----------+---------------------+-----------------------+----------------------+
|    5     |  0.9636  |    0:00:01.228715   |     0:00:00.000996    |    0:00:01.229710    |
|    15    |  0.7818  |    0:00:01.273662   |     0:00:00.001994    |    0:00:01.275656    |
|    30    |  0.8545  |    0:00:01.250818   |     0:00:00.002987    |    0:00:01.253806    |
|    45    |  0.8545  |    0:00:01.286563   |     0:00:00.003987    |    0:00:01.290550    |
| Average  |  0.8636  |    0:00:01.259940   |     0:00:00.002491    |    0:00:01.262431    |
| Std Dev. |  0.0649  | 0.02210947456528318 | 0.0011141953674541456 | 0.022970757309796638 |
+----------+----------+--

In [2]:
"test"

'test'